In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ..

/home/dmartins/GitWorkspace/phd-query-synthesis/PhDCoding


In [3]:
import sys, os
sys.path.append('..\\PhDCoding\\sada')
sys.path.append('..\\PhDCoding\\decision')
sys.path.append('..\\PhDCoding\\qbe')

In [4]:
from IPython.display import Image, HTML
from datamanagement.dataaccessobject import DataAccessObject, Dataset
from decision.somselector import SomSelector
from qbe.treeqbe import DecisionTreeQBE
from sada.decisionsada import DecisionSADA
from qbe import util
import pandas as pd, pandasql as pdsql

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

In [6]:
def show_images(indexes):
    for i in indexes:
        print("ID:{} - {}".format(i, dataset.original_data.iloc[i]['make']))
        display(Image(filename=(images[i]), width=300))

### Loading car dataset

In [7]:
DAO = DataAccessObject()
dataset = DAO.get_car_dataset()

/home/dmartins/GitWorkspace/phd-query-synthesis/PhDCoding/datamanagement/dataaccessobject.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.data_matrix = preprocessed_data.as_matrix()


### Configuring SADA

In [8]:
sada = DecisionSADA(dataset, selector=SomSelector(som_size=(10,10), num_iterations=1000))

In [9]:
images = ["car-images\\" + str(x) for x in dataset.original_data['imagepath']]

### Car dataset

In [10]:
HTML(dataset.original_data.to_html())

,make,manufacturer,type,price,mpg,num_of_cylinders,horsepower,fuel_tank_capacity,RPM,Wheelbase,Rear.seat.room,Weight,automatic_gearbox,passenger_capacity,length,width,luggage_capacity,AirBags,DriveTrain,Origin,imagepath
1,Acura Integra,Acura,Small,7950.0,28.0,4,140,13.2,6300,102,26.500000,2705,0,5,177,68,11.000000,None,Front,1,01acura_integra.jpg
2,Acura Legend,Acura,Midsize,16950.0,21.5,6,200,18.0,5500,115,30.000000,3560,0,5,195,71,15.000000,DriverAndPassenger,Front,1,02Acura_Legend.jpg
3,Audi 90,Audi,Compact,14550.0,23.0,6,172,16.9,5500,102,28.000000,3375,0,5,180,67,14.000000,DriverOnly,Front,1,03audi_100.jpg
4,Audi 100,Audi,Midsize,18850.0,22.5,6,172,21.1,5500,106,31.000000,3405,0,6,193,70,17.000000,DriverAndPassenger,Front,1,04audi_90.jpg
5,BMW 535i,BMW,Midsize,15000.0,26.0,4,208,21.1,5700,109,27.000000,3640,0,4,186,69,13.000000,DriverOnly,Rear,1,05bmw_535i.jpg
6,Buick Century,Buick,Midsize,7850.0,26.5,4,110,16.4,5200,105,28.000000,2880,1,6,189,69,16.000000,DriverOnly,Front,0,06buick_century.jpg
7,Buick LeSabre,Buick,Large,10400.0,23.5,6,170,18.0,4800,111,30.500000,3470,1,6,200,74,17.000000,DriverOnly,Front,0,07buick_lesabre.jpeg
8,Buick Roadmaster,Buick,Large,11850.0,20.5,6,180,23.0,4000,116,30.500000,4105,1,6,216,78,21.000000,DriverOnly,Rear,0,08buick_roadmaster.jpg
9,Buick Riviera,Buick,Midsize,13150.0,23.0,6,170,18.8,4800,108,26.500000,3495,1,5,198,73,14.000000,DriverOnly,Front,0,09buick_riviera.jpg
10,Cadillac DeVille,Cadillac,Large,17350.0,20.5,8,200,18.0,4100,114,35.000000,3620,1,6,206,73,18.000000,DriverOnly,Front,0,10Cadillac_DeVille.jpg


In [11]:
concept = dataset.original_data.query('type=="Sporty" and Origin ==0')
print(concept.index)
y_test = [int(y in concept.index) for y in range(1, dataset.original_data.shape[0]+1)]
print(y_test)

Int64Index([14, 19, 28, 34, 35, 60, 72, 75], dtype='int64')
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Select an example index to start the learning process

In [12]:
example_index = 33
tuple_input = dataset.data_matrix[example_index].tolist()
print(dataset.original_data.iloc[example_index].make)
##show_images([example_index])

Ford Mustang


### Selected examples

In [13]:
selected_indexes = sada.select(query=tuple_input)
print(selected_indexes)

[33, 18, 2, 13, 27]


In [14]:
predicted = [int(i in selected_indexes) for i in range(dataset.original_data.shape[0])]
print(predicted)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
precision_recall_fscore_support(y_test, predicted, average='binary')

(0.8, 0.5, 0.6153846153846154, None)

### Query learning phase

In [16]:
# Copying dataset to a dataframe
dataframe = dataset.preprocessed_data.copy(deep=True)

# Configure PandaSQL to query Pandas dataframe
pysql = lambda q: pdsql.sqldf(q, globals())

In [17]:
query_learner = DecisionTreeQBE(dataframe, selected_indexes)
best_predicate = query_learner.search_best_predicate()

In [18]:
learned_view = util.get_view_from_predicate(best_predicate, dataframe, dataframe.iloc[selected_indexes], pysql)

Query:  SELECT * FROM dataframe WHERE horsepower > 0.9897959232330322 OR horsepower <= 0.9897959232330322 AND Audi > 0.5 AND luggage_capacity <= 0.59375 OR horsepower <= 0.9897959232330322 AND Audi <= 0.5 AND Sporty > 0.5 AND length > 0.49358974397182465 AND length <= 0.6858974397182465
---------------------------------
Relevant & Actual:  5
Recall:  1.0
Specificity:  1.0
Precision:  1.0
F1-Score:  1.0
False negative rate:  0.0
False positive rate:  0.0
---------------------------------


### Show retrieved data

In [19]:
original_indexes = util.get_original_indexes_from_view(dataframe, learned_view)

In [20]:
HTML(dataset.original_data.iloc[original_indexes].to_html())

,make,manufacturer,type,price,mpg,num_of_cylinders,horsepower,fuel_tank_capacity,RPM,Wheelbase,Rear.seat.room,Weight,automatic_gearbox,passenger_capacity,length,width,luggage_capacity,AirBags,DriveTrain,Origin,imagepath
3,Audi 90,Audi,Compact,14550.0,23.0,6,172,16.9,5500,102,28.000000,3375,0,5,180,67,14.000000,DriverOnly,Front,1,03audi_100.jpg
14,Chevrolet Camaro,Chevrolet,Sporty,7550.0,23.5,6,160,15.5,4600,101,25.000000,3240,0,4,193,74,13.000000,DriverAndPassenger,Rear,0,14chevrolet_camaro.jpg
19,Chevrolet Corvette,Chevrolet,Sporty,19000.0,21.0,8,300,20.0,5000,96,30.901059,3380,0,2,179,74,17.203669,DriverOnly,Rear,0,19chevrolet_corvette.jpeg
28,Dodge Stealth,Dodge,Sporty,12900.0,21.0,6,300,19.8,6000,97,20.000000,3805,0,4,180,72,11.000000,DriverOnly,4WD,0,28dodge_stealth.jpg
34,Ford Mustang,Ford,Sporty,7950.0,25.5,4,105,15.4,4600,101,24.000000,2850,0,4,180,68,12.000000,DriverOnly,Rear,0,34ford_mustang.jpg


In [24]:
queries = ["type == 'Sporty' and Origin == 0", 
    "type != 'Sporty' and Origin == 1",
    "automatic_gearbox == 1 and horsepower >= 150",
    "luggage_capacity >= 18 and passenger_capacity > 5",
    "price <= 7000 and mpg >= 26 and automatic_gearbox == 0",
    "manufacturer == 'Ford' or manufacturer == 'Chevrolet'"]

In [55]:
results = []
for q in queries:
    concept = dataset.original_data.query(q)
    y_test = [int(y in concept.index) for y in range(1, dataset.original_data.shape[0]+1)]
    sada = DecisionSADA(dataset, selector=SomSelector(som_size=(6,6), num_iterations=1000))
    for i in range(10):
        example_index = random.choice(concept.index)
        tuple_input = dataset.data_matrix[example_index-1].tolist()
        selected_indexes = sada.select(query=tuple_input,  num_of_selected_candidates=len(concept))
        predicted = [int(i in selected_indexes) for i in range(dataset.original_data.shape[0])]
        res = [q, len(c.index)] + list(precision_recall_fscore_support(y_test, predicted, average='binary'))[:3]
        results.append(res)

In [58]:
df = pd.DataFrame(results, columns=['query', 'concept size', 'precision', 'recall', 'f1score'])

In [64]:
df.groupby(by=['query', 'concept size']).mean().sort_values(by='f1score', ascending=False)

,,precision,recall,f1score
query,concept size,,,
query_id_type == 'Sporty' and Origin == 0,concept_8,0.700000,0.700000,0.700000
query_id_price <= 7000 and mpg >= 26 and automatic_gearbox == 0,concept_25,0.660000,0.660000,0.660000
query_id_type != 'Sporty' and Origin == 1,concept_39,0.443590,0.443590,0.443590
query_id_automatic_gearbox == 1 and horsepower >= 150,concept_23,0.395652,0.395652,0.395652
query_id_luggage_capacity >= 18 and passenger_capacity > 5,concept_9,0.333333,0.333333,0.333333
query_id_manufacturer == 'Ford' or manufacturer == 'Chevrolet',concept_16,0.268750,0.268750,0.268750
